In [1]:
import numpy as np
import pandas as pd
from CoolProp.CoolProp import PropsSI, PhaseSI

In [2]:

def get_sat_pressure(T_hot, work_fluid="Water"):
    """ Gets the saturation pressure in bar of a fluid at a specified temp in celcius """
    T_hot += 273.15
    P_sat = PropsSI('P', 'T', T_hot, 'Q', 0, work_fluid)
    return P_sat/1e5



def simulate_rankine(T_hot, T_cold, P_high_bar, work_fluid="Water"):
    """ Simulate a Rankine cycle with defined hot and cold values (celcius), 
    high pressure (bar), and working fluid. Returns a Dataframe design table. """
    T_hot += 273.15
    T_cold += 273.15
    P_high = P_high_bar * 1e5
    
    # 1: Boiler Outlet
    T_1 = T_hot
    P_1 = P_high
    H_1 = PropsSI('H', 'T', T_1, 'P', P_1, work_fluid)
    S_1 = PropsSI('S', 'T', T_1, 'P', P_1, work_fluid)
    phase_1  = PhaseSI('T', T_1, 'P', P_1, work_fluid)

    # 2: Turbine Outlet
    P_2 = PropsSI('P', 'T', T_cold, 'Q', 0, work_fluid) # we want saturation pressure at outlet
    S_2 = S_1
    H_2 = PropsSI('H', 'P', P_2, 'S', S_2, work_fluid)
    T_2 = PropsSI('T', 'P', P_2, 'S', S_2, work_fluid)
    X_2 = PropsSI('Q', 'P', P_2, 'S', S_2, work_fluid)
    phase_2  = PhaseSI('P', P_2, 'S', S_2, work_fluid)

    # 3: Condenser Outlet
    T_3 = T_cold
    X_3 = 0 # force a saturated liquid
    H_3 = PropsSI('H', 'T', T_3, 'Q', X_3, work_fluid)
    P_3 = PropsSI('P', 'T', T_3, 'Q', X_3, work_fluid)
    S_3 = PropsSI('S', 'T', T_3, 'Q', X_3, work_fluid)
    phase_3  = PhaseSI('T', T_3, 'Q', X_3, work_fluid)

    # 4: Pump outlet
    P_4 = P_high
    S_4 = S_3
    H_4 = PropsSI('H', 'P', P_4, 'S', S_4, work_fluid)
    T_4 = PropsSI('T', 'P', P_4, 'S', S_4, work_fluid)
    phase_4  = PhaseSI('P', P_4, 'S', S_4, work_fluid)

    # Create DataFrame
    states = [
        ['1', phase_1, P_1/1e5, T_1-273.15, '',  H_1/1000, S_1/1000],
        ['2', phase_2, P_2/1e5, T_2-273.15, X_2 if 0 <= X_2 <= 1 else '', H_2/1000, S_2/1000],
        ['3', phase_3, P_3/1e5, T_3-273.15, X_3, H_3/1000, S_3/1000],
        ['4', phase_4, P_4/1e5, T_4-273.15, '',  H_4/1000, S_4/1000],
    ]
    
    df = pd.DataFrame(states, columns=[
        'State', 'Phase', 'Pressure (Bar)', 'Temperature (C)', 'Quality', 
        'Enthalpy (kJ/kg)', 'Entropy (kJ/kg-K)'
    ])
    
    return df 



def get_required_mass_flow(df, target_power_kW=25000):
    """ Returns the mass flow rate (in kg/s) required to meet a power requirement (default 25 MW) """
    if df is None:
        return -1
        
    h1 = df.loc[df['State'] == '1', 'Enthalpy (kJ/kg)'].values[0]
    h2 = df.loc[df['State'] == '2', 'Enthalpy (kJ/kg)'].values[0]
    
    delta_h = h1 - h2
    
    return target_power_kW / delta_h



def get_eta(df):
    """ Returns the efficiency (eta) of a rankine cycle """
    if df is None:
        return -1

    h1 = df.loc[df['State'] == '1', 'Enthalpy (kJ/kg)'].values[0]
    h2 = df.loc[df['State'] == '2', 'Enthalpy (kJ/kg)'].values[0]
    h4 = df.loc[df['State'] == '4', 'Enthalpy (kJ/kg)'].values[0]

    return (h1 - h2) / (h1 - h4)


In [3]:

# In our preliminary design report, we made a cycle between 25C and 200C with a pressure of 10 bar. Try it here:
T_hot = 200
T_cold = 25

df = simulate_rankine(T_hot, T_cold, 10)

mass_flow = get_required_mass_flow(df)
eta = get_eta(df)

print(f"Mass Flow: {mass_flow:.2f} kg/s")
print(f"eta: {eta:.4f}")


Mass Flow: 29.88 kg/s
eta: 0.3073


### This is the exact design table and efficiency we got from Aspen, so I am fairly confident this will work!

In [4]:

def optimize_rankine_cycle(work_fluid):
    """ Optimizes a Rankine cycle for a specific fluid by brute-forcing temperatures and max pressure. """
    
    best_eta = -1
    best_params = None
    best_df = None
    
    # Search bounds from low temp to high temp
    T_cold_range = np.linspace(25, 200, 50)
    T_hot_range = np.linspace(25, 200, 50) 

    for T_cold in T_cold_range:
        for T_hot in T_hot_range:

            # there is a better way of doing this!
            if T_hot <= T_cold:
                continue 

            try:
                
                P_sat = get_sat_pressure(T_hot, work_fluid)

                # Try a range of values from 0.1 bar to rrrigghhttt under P_sat to avoid coolprop sadness
                for P_high_bar in np.linspace(0.1, 0.999 * P_sat, 20):
                    try:
                        df = simulate_rankine(T_hot, T_cold, P_high_bar, work_fluid)
                        eta = get_eta(df)

                        # check if we are in two-phase region -- while possible to have a gas coming out of turbine, this fits the scope of the assignment better
                        phase_2 = df.loc[df['State'] == '2', 'Phase'].values[0]
                        if phase_2 == 'twophase' and eta > best_eta:
                            best_eta = eta
                            best_params = (T_cold, T_hot, P_high_bar)
                            best_df = df
                            
                    except:
                        continue  # coolprop throws lots of helpful errors when performing transitions that don't exist
            except:
                continue

    return best_eta, best_params, best_df


def print_best_params(work_fluid):
    """ Prints the best parameters for a working fluid """
    print(f"{work_fluid}: ")
    best_eta, best_params, best_df = optimize_rankine_cycle("Water")
    print(f"Best eta: {best_eta:.4f}")
    print(f"Best Params: {best_params}")
    print(f"Required flowrate (kg/s): {get_required_mass_flow(best_df)}")
    best_df



In [5]:
print("Water:")
best_eta, best_params, best_df = optimize_rankine_cycle("Water")
print(f"Best eta: {best_eta:.4f}")
print(f"Best Params (T_cold, T_hot, P): {best_params}")
print(f"Required flowrate (kg/s): {get_required_mass_flow(best_df)}")
best_df

Water:
Best eta: 0.3274
Best Params (T_cold, T_hot, P): (25.0, 200.0, 15.53372972568236)
Required flowrate (kg/s): 28.427415125545213


,State,Phase,Pressure (Bar),Temperature (C),Quality,Enthalpy (kJ/kg),Entropy (kJ/kg-K)
0,1,gas,15.533730,200.000000,,2792.121161,6.430814
1,2,twophase,0.031699,25.000000,0.740417,1912.688407,6.430814
2,3,twophase,0.031699,25.000000,0,104.829222,0.367225
3,4,liquid,15.533730,25.028629,,106.383546,0.367225


In [6]:
print("Toluene:")
best_eta, best_params, best_df = optimize_rankine_cycle("Toluene")
print(f"Best eta: {best_eta:.4f}")
print(f"Best Params (T_cold, T_hot, P): {best_params}")
print(f"Required flowrate (kg/s): {get_required_mass_flow(best_df)}")
best_df

Toluene:
Best eta: 0.1517
Best Params (T_cold, T_hot, P): (25.0, 85.71428571428572, 0.4712414170334203)
Required flowrate (kg/s): 339.0131214695533


,State,Phase,Pressure (Bar),Temperature (C),Quality,Enthalpy (kJ/kg),Entropy (kJ/kg-K)
0,1,gas,0.471241,85.714286,,327.839865,0.918354
1,2,twophase,0.037993,25.000000,0.998749,254.096407,0.918354
2,3,twophase,0.037993,25.000000,0,-158.239630,-0.464627
3,4,liquid,0.471241,25.009523,,-158.189379,-0.464627


In [7]:
print("R245fa:")
best_eta, best_params, best_df = optimize_rankine_cycle("R245fa")
print(f"Best eta: {best_eta:.4f}")
print(f"Best Params: {best_params}")
print(f"Required flowrate (kg/s): {get_required_mass_flow(best_df)}")
best_df

R245fa:
Best eta: 0.2114
Best Params: (25.0, 153.57142857142858, 36.281980189223596)
Required flowrate (kg/s): 496.7931269440753


,State,Phase,Pressure (Bar),Temperature (C),Quality,Enthalpy (kJ/kg),Entropy (kJ/kg-K)
0,1,gas,36.281980,153.571429,,473.609254,1.753795
1,2,twophase,1.485811,25.000000,0.995183,423.286496,1.753795
2,3,twophase,1.485811,25.000000,0,232.982608,1.115513
3,4,liquid,36.281980,26.173984,,235.574950,1.115513


In [9]:
print("R1233zdE:")
best_eta, best_params, best_df = optimize_rankine_cycle("R1233zdE")
print(f"Best eta: {best_eta:.4f}")
print(f"Best Params: {best_params}")
print(f"Required flowrate (kg/s): {get_required_mass_flow(best_df)}")
best_df

R1233zdE:
Best eta: 0.2270
Best Params: (25.0, 164.2857142857143, 34.89524235419753)
Required flowrate (kg/s): 454.07812104123536


,State,Phase,Pressure (Bar),Temperature (C),Quality,Enthalpy (kJ/kg),Entropy (kJ/kg-K)
0,1,gas,34.895242,164.285714,,506.730792,1.867932
1,2,twophase,1.298069,25.000000,0.994878,451.674186,1.867932
2,3,twophase,1.298069,25.000000,0,261.498598,1.230080
3,4,liquid,34.895242,26.230439,,264.151831,1.230080


In [10]:
print("Cyclopentane:")
best_eta, best_params, best_df = optimize_rankine_cycle("Cyclopentane")
print(f"Best eta: {best_eta:.4f}")
print(f"Best Params: {best_params}")
print(f"Required flowrate (kg/s): {get_required_mass_flow(best_df)}")
best_df

Cyclopentane:
Best eta: 0.1373
Best Params: (25.0, 78.57142857142858, 2.423088636177762)
Required flowrate (kg/s): 386.9643960604074


,State,Phase,Pressure (Bar),Temperature (C),Quality,Enthalpy (kJ/kg),Entropy (kJ/kg-K)
0,1,gas,2.423089,78.571429,,425.401250,1.216319
1,2,twophase,0.423444,25.000000,0.998932,360.795824,1.216319
2,3,twophase,0.423444,25.000000,0,-45.433874,-0.146182
3,4,liquid,2.423089,25.059465,,-45.163825,-0.146182


In [11]:
print("n-Pentane")
best_eta, best_params, best_df = optimize_rankine_cycle("nPentane")
print(f"Best eta: {best_eta:.4f}")
print(f"Best Params: {best_params}")
print(f"Required flowrate (kg/s): {get_required_mass_flow(best_df)}")
best_df

n-Pentane
Best eta: -1.0000
Best Params: None
Required flowrate (kg/s): -1


In [12]:
print("Ammonia")
best_eta, best_params, best_df = optimize_rankine_cycle("Ammonia")
print(f"Best eta: {best_eta:.4f}")
print(f"Best Params: {best_params}")
print(f"Required flowrate (kg/s): {get_required_mass_flow(best_df)}")
best_df

Ammonia
Best eta: 0.2158
Best Params: (25.0, 132.14285714285717, 107.04179499050417)
Required flowrate (kg/s): 114.58221384400606


,State,Phase,Pressure (Bar),Temperature (C),Quality,Enthalpy (kJ/kg),Entropy (kJ/kg-K)
0,1,gas,107.041795,132.142857,,1490.149752,4.604457
1,2,twophase,10.026950,25.000000,0.693755,1271.965802,4.604457
2,3,twophase,10.026950,25.000000,0,463.174878,1.891759
3,4,liquid,107.041795,27.479075,,479.194443,1.891759


In [13]:
print("isobutane")
best_eta, best_params, best_df = optimize_rankine_cycle("isobutane")
print(f"Best eta: {best_eta:.4f}")
print(f"Best Params: {best_params}")
print(f"Required flowrate (kg/s): {get_required_mass_flow(best_df)}")
best_df

isobutane
Best eta: 0.1630
Best Params: (46.42857142857143, 132.14285714285717, 34.76290918288151)
Required flowrate (kg/s): 431.61841510153926


,State,Phase,Pressure (Bar),Temperature (C),Quality,Enthalpy (kJ/kg),Entropy (kJ/kg-K)
0,1,gas,34.762909,132.142857,,673.449000,2.325179
1,2,twophase,6.266775,46.428571,0.997776,615.527468,2.325179
2,3,twophase,6.266775,46.428571,0,312.763337,1.377793
3,4,liquid,34.762909,48.208168,,318.189142,1.377793


## Next, let's optimize the original cycle for better quality:

In [51]:

# We want >0.92 quality with water
T_vals = np.linspace(150, 200, 100)
P_fracs = np.linspace(0.01, 0.99, 100)

df_list = []
eta_list = []
mass_flow_list = []
pressure_frac_list = []

for T_hot in T_vals:

    P_sat = get_sat_pressure(T_hot, "Water")
    
    for P_high in P_fracs:
        
        try:
            df = simulate_rankine(T_hot, 25, (P_sat * P_high), "Water")
            

            if df.loc[df['State'] == '2', 'Quality'].values[0] > 0.92:
            
                df_list.append(df)
                eta_list.append(get_eta(df))
                mass_flow_list.append(get_required_mass_flow(df))
                pressure_frac_list.append(P_high)
        
        except:
            continue


idx = mass_flow_list.index(min(mass_flow_list))
print(f"Min mass flow: {mass_flow_list[idx]:.3f} kg/s" )
print(f"Eta: {eta_list[idx]:.4f} ")
df_list[idx]

Min mass flow: 48.690 kg/s
Eta: 0.1860 


,State,Phase,Pressure (Bar),Temperature (C),Quality,Enthalpy (kJ/kg),Entropy (kJ/kg-K)
0,1,gas,0.822375,194.444444,,2865.341246,7.903890
1,2,twophase,0.031699,25.000000,0.920293,2351.885846,7.903890
2,3,twophase,0.031699,25.000000,0,104.829222,0.367225
3,4,liquid,0.822375,25.001455,,104.908526,0.367225


## Optimize with Ammonia:

In [53]:

# We want >0.92 quality with Ammonia as well 
T_vals = np.linspace(50, 200, 100)
P_fracs = np.linspace(0.01, 0.99, 100)

df_list = []
eta_list = []
mass_flow_list = []
pressure_frac_list = []

for T_hot in T_vals:

    try:
    
        P_sat = get_sat_pressure(T_hot, "Ammonia")

    except:
        continue
    
    for P_high in P_fracs:
        
        try:
            df = simulate_rankine(T_hot, 25, (P_sat * P_high), "Ammonia")
            

            if df.loc[df['State'] == '2', 'Quality'].values[0] > 0.92:
            
                df_list.append(df)
                eta_list.append(get_eta(df))
                mass_flow_list.append(get_required_mass_flow(df))
                pressure_frac_list.append(P_high)
        
        except:
            continue


idx = mass_flow_list.index(min(mass_flow_list))
print(f"Min mass flow: {mass_flow_list[idx]:.3f} kg/s" )
print(f"Eta: {eta_list[idx]:.4f} ")
df_list[idx]

Min mass flow: 108.013 kg/s
Eta: 0.1780 


,State,Phase,Pressure (Bar),Temperature (C),Quality,Enthalpy (kJ/kg),Entropy (kJ/kg-K)
0,1,gas,56.779842,131.818182,,1771.513092,5.503649
1,2,twophase,10.026950,25.000000,0.923718,1540.060079,5.503649
2,3,twophase,10.026950,25.000000,0,463.174878,1.891759
3,4,liquid,56.779842,26.211724,,470.912211,1.891759
